In [52]:
import numpy as np
import pandas as pd
import matplotlib as plt
import bs4 as bs
import requests
from lxml import html
from pprint import pprint
import random

__Дизайн взаимодействий user-item__


`1)` Работаем пока только с иностранными компаниями (USA)

`2)` Облигации и ETF тоже вычеркиваем из списка покупок

`3)` Будем выбирать по несколько акций (3-5) в месяц из списка популярных

`4)` Начальные условия: пусть раз в неделю пользователь совершает одну сделку на покупку актива из списка `spb_list`. UPD: После первого захода на бейзлайны я пришел к выводу, что список нужно расширять, иначе ничего не остается рекомендовать. Теперь пусть пользователи совершают хотя бы одну сделку из всего списка `finviz.` 

`5)` Времянные рамки: 24 недели. Гарантированно, что пользователь сделает хотя бы одну покупку за 4 недели. 

__Фиксация покупок ценных бумаг по неделям__

In [53]:
id_list = np.arange(1, 100001, 1)

In [54]:
data=dict()

In [55]:
#Сгенерируем номера недель когда пользователь совершает покупки ценных бумаг
for i in range(len(id_list)):
    min_week=28
    max_week=52
    step=4
    temp=[]
    while min_week!=max_week: 
        step=4
        spam = random.randrange(min_week, min_week+step)
        temp.append(spam)
        min_week = min_week+step
    data[id_list[i]] = temp

In [56]:
#из словаря составим серию
data = pd.Series(data)

In [57]:
data

1         [31, 32, 39, 43, 47, 49]
2         [28, 33, 37, 42, 47, 48]
3         [28, 33, 38, 42, 46, 48]
4         [31, 35, 39, 41, 44, 48]
5         [28, 35, 36, 41, 45, 49]
                    ...           
99996     [28, 32, 36, 40, 46, 50]
99997     [31, 32, 39, 43, 46, 51]
99998     [29, 32, 38, 40, 44, 48]
99999     [28, 34, 37, 42, 47, 49]
100000    [29, 35, 38, 42, 45, 51]
Length: 100000, dtype: object

In [58]:
df = pd.DataFrame({'week': data}).reset_index()

In [59]:
df.head()

,index,week
0,1,"[31, 32, 39, 43, 47, 49]"
1,2,"[28, 33, 37, 42, 47, 48]"
2,3,"[28, 33, 38, 42, 46, 48]"
3,4,"[31, 35, 39, 41, 44, 48]"
4,5,"[28, 35, 36, 41, 45, 49]"


In [60]:
df = df.rename(columns={'index': 'user_id'})

In [61]:
df.head()

,user_id,week
0,1,"[31, 32, 39, 43, 47, 49]"
1,2,"[28, 33, 37, 42, 47, 48]"
2,3,"[28, 33, 38, 42, 46, 48]"
3,4,"[31, 35, 39, 41, 44, 48]"
4,5,"[28, 35, 36, 41, 45, 49]"


In [62]:
# "стакаем" номера недель 
s = df.apply(lambda x: pd.Series(x['week']), axis=1).stack().reset_index(level=1, drop=True)

In [63]:
s.name = 'week_no'
df = df.drop('week', axis=1).join(s)

In [64]:
#проверка
df = df.reset_index()

In [65]:
df = df.drop('index', axis=1)

In [66]:
df.head(15)

,user_id,week_no
0,1,31
1,1,32
2,1,39
3,1,43
4,1,47
5,1,49
6,2,28
7,2,33
8,2,37
9,2,42


In [67]:
df.shape

(600000, 2)

__Забираем данные из СПБ биржи__

In [68]:
shares_df = pd.read_csv('spb100.csv')

In [69]:
shares_df.head()

,tickers,name_company,ISIN
0,BABA,Alibaba Group Holding Limited,US01609W1027
1,AAPL,Apple Inc.,US0378331005
2,TSLA,"Tesla, Inc.",US88160R1014
3,SPCE,"Virgin Galactic Holdings, Inc.",US92766K1060
4,AMZN,"Amazon.com, Inc.",US0231351067


In [70]:
isin_tickers = shares_df.ISIN.unique()

Отфильтруем по ISIN только те, которые торгуются на USA. 

In [71]:
shares_df = shares_df.loc[shares_df['ISIN'].str.contains('US', case=False)]
shares_df = shares_df.loc[~shares_df['tickers'].str.contains("@", case=False)]

In [72]:
shares_df.head()

,tickers,name_company,ISIN
0,BABA,Alibaba Group Holding Limited,US01609W1027
1,AAPL,Apple Inc.,US0378331005
2,TSLA,"Tesla, Inc.",US88160R1014
3,SPCE,"Virgin Galactic Holdings, Inc.",US92766K1060
4,AMZN,"Amazon.com, Inc.",US0231351067


Удалим _RDS A_ из списка тикеров, потому что он уже не торгуется

In [73]:
shares_df = shares_df.drop(shares_df[shares_df.tickers == 'RDS A'].index)

In [74]:
shares_df

,tickers,name_company,ISIN
0,BABA,Alibaba Group Holding Limited,US01609W1027
1,AAPL,Apple Inc.,US0378331005
2,TSLA,"Tesla, Inc.",US88160R1014
3,SPCE,"Virgin Galactic Holdings, Inc.",US92766K1060
4,AMZN,"Amazon.com, Inc.",US0231351067
...,...,...,...
95,FDX,FedEx Corporation,US31428X1063
96,PINS,"Pinterest, Inc. ClassA",US72352L1061
97,OKE,"ONEOK, Inc.",US6826801036
98,TSN,"Tyson Foods, Inc. Class A",US9024941034


In [75]:
shares_df.shape

(91, 3)

__"Приклеим" сразу же данные из finviz: цену акции, ee id__

In [76]:
finviz_df = pd.read_csv('finviz_shares.csv')

In [77]:
finviz_df_ticker = finviz_df[['ticker', 'id_share', 'Price']]

In [78]:
finviz_df_ticker = finviz_df_ticker.rename(columns={'ticker': 'tickers'})

In [79]:
pd.merge(shares_df, finviz_df_ticker, how="inner", on="tickers")

,tickers,name_company,ISIN,id_share,Price
0,BABA,Alibaba Group Holding Limited,US01609W1027,706,123.98
1,AAPL,Apple Inc.,US0378331005,4,172.12
2,TSLA,"Tesla, Inc.",US88160R1014,7664,904.55
3,SPCE,"Virgin Galactic Holdings, Inc.",US92766K1060,7111,8.85
4,AMZN,"Amazon.com, Inc.",US0231351067,388,3180.07
...,...,...,...,...,...
85,FDX,FedEx Corporation,US31428X1063,2757,240.41
86,PINS,"Pinterest, Inc. ClassA",US72352L1061,5996,25.55
87,OKE,"ONEOK, Inc.",US6826801036,5604,63.02
88,TSN,"Tyson Foods, Inc. Class A",US9024941034,7661,97.88


In [80]:
shares_spb = pd.merge(shares_df, finviz_df_ticker, how="inner", on="tickers")

In [102]:
shares_spb.to_csv('popular_spb.csv', encoding='utf-8', index=False)

_UPD: и все-таки с spb акциями не получается хуже. Лучше взять всю выборку из finviz_

In [81]:
id_shares = finviz_df.id_share.to_list()

In [85]:
data_share=dict()
week_amount = 6

In [86]:
for i in range(len(id_list)): 
    general_shares = []
    for j in range(week_amount): 
        amount_share = random.randint(1, 6)
        shares = random.sample(id_shares, k=amount_share)
        general_shares.append(shares)
    data_share[id_list[i]] = general_shares

In [87]:
#data_share

In [88]:
#из словаря составим серию
data_share = pd.Series(data_share)

In [89]:
data_share_df = pd.DataFrame({'shares': data_share}).reset_index()

In [90]:
data_share_df.head()

,index,shares
0,1,"[[3375, 1810], [322, 3965], [2816, 7203, 2227]..."
1,2,"[[1317, 7561], [8042, 3448, 3998], [4991, 7989..."
2,3,"[[227, 7286, 5394], [2192, 2441], [1730, 3960,..."
3,4,"[[3784, 8382, 4785], [2552], [3424, 7858, 4646..."
4,5,"[[1792], [146, 1360, 4476], [5783, 3715, 3429]..."


In [91]:
data_share_df.shape

(100000, 2)

In [92]:
# "стакаем" акции
f = data_share_df.apply(lambda x: pd.Series(x['shares']), axis=1).stack().reset_index(level=1, drop=True)
f.name = 'shares_deal'

In [93]:
data_share_df = data_share_df.drop('shares', axis=1).join(f)

In [94]:
data_share_df = data_share_df.reset_index()

In [95]:
data_share_df.shape

(600000, 3)

In [96]:
temp_list = pd.Series(data_share_df['shares_deal'])

In [97]:
df = pd.concat([df, temp_list], axis=1)

In [98]:
df.head(15)

,user_id,week_no,shares_deal
0,1,31,"[3375, 1810]"
1,1,32,"[322, 3965]"
2,1,39,"[2816, 7203, 2227]"
3,1,43,[3497]
4,1,47,[8337]
5,1,49,"[689, 1039, 6724, 7819]"
6,2,28,"[1317, 7561]"
7,2,33,"[8042, 3448, 3998]"
8,2,37,"[4991, 7989, 8086]"
9,2,42,"[194, 6181]"


In [99]:
file_name = 'interactions.csv'

In [100]:
df.to_csv(file_name, encoding='utf-8', index=False)